# RDS connect

In [ ]:
#################################
#             안내사항            #
#################################

# 1. pymysql을 이용하는 방법입니다.
# 2. 필요한 모든 세팅을 아래 코드에 했으므로 바로 이용하시면 됩니다.
# 3. 문제가 발생할 경우 박종혁에게 연락 부탁드립니다.


#################################
#             주의사항            #
#################################

# 1. 작업이 끝나면 반드시 연결을 닫아주세요. (conn.close())
# 2. 한글 입력이 가능하도록 설정을 변경했습니다. (RDS - 파라미터그룹 - utf8mb4)
# 3. 새로 생성되는 테이블의 CHAR SET 은 UTF-8로 설정해주세요.

In [53]:
%pip install pymysql
import pymysql
import pandas as pd

In [39]:
# 연결 RDS 정보

host = "team06-antifragile-db.cxuncqkdvk3h.us-east-1.rds.amazonaws.com"
port = 3306
username = "admin"
database = "antifragile"
password = "antifragile1234"

In [87]:
# pymysql을 통한 로컬 - AWS RDS 연결

conn = pymysql.connect(host = host, user = username, port = port, database = database, password = password)
cursor = conn.cursor()

In [ ]:
# 명령문 실행

## 명령문 작성
sql = '''
show tables;
'''

## 명령문 실행
cursor.execute(sql)

## 실행 결과 출력
response = cursor.fetchall()
for data in response:
    print(data)

In [89]:
# sql 연결 해제

conn.close()

# S3 connect

In [ ]:
#################################
#             안내사항            #
#################################

# 1. boto3를 이용하는 방법입니다.
# 2. 사용자 계정 세팅은 직접 해주셔야 합니다. 아래 주의사항을 참고해주세요.
# 3. 그 외의 사항들은 바로 사용할 수 있게 세팅하였습니다.
# 4. 문제 발생시 박종혁에게 연락해주세요.


#################################
#             주의사항            #
#################################

# 계정은 각자의 AWS 계정에 해당하는 access_key_id와 aws_secret_accesskey 를 사용해주세요.
# 두 가지는 처음 AWS 사용시에 직접 발급받으셨을 겁니다. (강사님께서 절대 까먹지 말라고 하신 것)

# 계정을 본 코드에 직접 노출하는 것은 추천하지 않습니다.
# AWS CLI 세팅을 통해 미리 access_key_id와 aws_secret_accesskey를 세팅해주신 뒤
# 본 코드에는 키가 노출되지 않도록 해주시기 바랍니다.

In [ ]:
%pip install boto3
import boto3
import os
import pandas as pd

In [2]:
# 계정 설정 (직접 세팅)
# CLI 로 미리 세팅을 해놓기를 바랍니다. 불가능할 경우 아래에 키를 넣어주세요. (하지만 추천하지 않음)

# aws_access_key_id = pd.read_csv('키파일.csv').iloc[0].loc['Access key ID']
# aws_secret_access_key = pd.read_csv('키파일.csv').iloc[0].loc['Secret access key']


In [101]:
# s3 정보 입력

resource = 's3'
bucket_name = 'team06-antifragile-s3'
region_name = 'us-east-1'

## boto3.resource() 사용시

boto3를 이용하는 방법은 두 가지가 있습니다.  
1. boto3.resource() 를 사용하는 방법  
2. boto3.client() 를 사용하는 방법  

먼저, boto3.resource() 메서드를 사용하는 방법을 아래에서 다룹니다.  
둘의 차이점은 boto3.client() 설명부에서 다루겠습니다.

★★★★★ boto3.client() 방식은 ec2 자동생성 오류때문에 추천하지 않습니다. ★★★★★

In [102]:
# s3 접속 resource 선언
try:
    s3 = boto3.resource(resource, aws_access_key_id = aws_access_key_id,
                    aws_secret_access_key = aws_secret_access_key,
                    region_name = region_name)
    # 계정을 직접 입력할 경우
except:
    s3 = boto3.resource('s3')
    # CLI 세팅을 했을 경우

In [109]:
# 버킷에 업로드되어있는 파일 확인

buckets = s3.Bucket(name = bucket_name)

for obj in buckets.objects.all():
    print(obj)


s3.ObjectSummary(bucket_name='team06-antifragile-s3', key='앞으로의 프로젝트.xlsx')


In [104]:
# 버킷에 파일 업로드 (방법 1)
# s3.meta.client.upload_file('local/file/path', 'bucket_name', 'bucket/file/path')

s3.meta.client.upload_file('./antifragile.sql', bucket_name, 'test.sql')

In [49]:
# 버킷에서 파일 다운로드 (방법 1)
# s3.meta.client.download_file('bucket_name', 'file_key', 'local/file/path')
# file_key = bucket/file/path

s3.meta.client.download_file(bucket_name, 'test.sql', '/Users/jongya/Desktop/test2.sql')

In [ ]:
# 버킷에서 파일 삭제
# s3.Object(bucket_name, file_key).delete()

s3.Object(bucket_name, 'test.sql').delete()

In [70]:
# 버킷에서 특정 문자가 포함된 파일 삭제

buckets = s3.Bucket(name = bucket_name)

for i in buckets.objects.all():
    if 'test' in i.key:
        s3.Object(bucket_name, i.key).delete()

## boto3.client() 사용시

**★★★★★★주의★★★★★★**

boto3.client() 방식을 사용할 때에는 주의하시기 바랍니다.  
run_instances loop에 빠질 경우 EC2가 자동적으로 생성될 수 있으므로  
되도록이면 연결 방식은 boto3.resource 를 사용하시기 바랍니다.  

만약 굳이 boto3.client() 방식 연결을 해야할 경우,  
가장 하단의 boto3.client의 session configuration 섹션을 참고하여 코딩하시기 바랍니다.

boto3.resource()와 boto3.client()의 차이

boto3의 boto3.resource() 및 boto3.client() 함수는 AWS 서비스와 상호 작용하기 위한 다양한 인터페이스를 제공합니다.

boto3.resource() 함수는 AWS 서비스에 대한 상위 수준 객체 지향 인터페이스를 반환합니다. 이 인터페이스는 S3 버킷 또는 EC2 인스턴스와 같은 AWS 리소스를 나타내는 리소스 객체 세트를 제공하고 이러한 리소스와 보다 자연스러운 Python 방식으로 상호 작용할 수 있는 메서드 세트를 노출합니다. 예를 들어 s3.Object(bucket_name, key).put() 또는 s3.Object(bucket_name, key).get()과 같은 메서드를 사용하여 S3 객체에 대한 작업을 수행할 수 있습니다.

반면 boto3.client() 함수는 AWS 서비스에 대한 하위 수준 인터페이스를 반환하여 AWS API 주변에 얇은 래퍼를 제공합니다. 이 인터페이스는 AWS API 작업에 직접 해당하는 일련의 메서드를 노출하며 필요한 모든 파라미터를 명시적으로 지정해야 합니다. 예를 들어 s3.get_object(Bucket=bucket_name, Key=key) 또는 s3.put_object(Bucket=bucket_name, Key=key, Body=object_data)와 같은 메서드를 사용하여 S3 객체에 대한 작업을 수행할 수 있습니다.

일반적으로 AWS 리소스에 대한 보다 Python적인 객체 지향 인터페이스를 원하는 경우 boto3.resource()를 사용하고, AWS API 작업에 대한 세밀한 제어가 필요한 경우 boto3.client()를 사용해야 합니다.

-- chatgpt

In [59]:
# s3 클라이언트 선언

s3_client = boto3.client('s3', aws_access_key_id = aws_access_key_id,
                         aws_secret_access_key = aws_secret_access_key)

In [ ]:
# 버킷에 업로드 되어있는 파일 확인

res = s3_client.list_objects_v2(Bucket = bucket_name)

for obj in res['Contents']:
    print(obj['Key'])

In [72]:
# 버킷에 파일 업로드 (방법 2)
# s3_client.upload_file('/local/file/path', 'bucket_name', 'file_key')
# file_key = bucket/file/path

s3_client.upload_file('/Users/jongya/Desktop/antifragile.sql', bucket_name, 'test2.sql')

In [62]:
# 버킷에서 파일 다운로드 (방법 2)
# s3_client.download_file('bucket_name', 'file_key', 'local/file/path')
# file_key = bucket/file/path

s3_client.download_file(bucket_name, 'test.sql', '/Users/jongya/Desktop/test3.sql')

In [75]:
# 버킷에서 파일 삭제
# s3_client.delete_object(Bucket=bucket_name, Key=object_key)

s3_client.delete_object(Bucket = bucket_name, Key = 'test2.sql')

{'ResponseMetadata': {'RequestId': 'KE84D250HSDX5N26',
  'HostId': 'oj0o5QEzDo5LU2dVILH3ASAu0GoFutMtIHa0DziP2aRPClgv6EOjzlsKAw8wpd/eYdOQfTZlpn0=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'oj0o5QEzDo5LU2dVILH3ASAu0GoFutMtIHa0DziP2aRPClgv6EOjzlsKAw8wpd/eYdOQfTZlpn0=',
   'x-amz-request-id': 'KE84D250HSDX5N26',
   'date': 'Sun, 05 Mar 2023 10:55:16 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Prefix 의 개념

S3에는 객체 스토리지 서비스로, 전통적인 directory 개념이 없다.  
개체는 버킷으로 구성되고 고유 키(경로)로 식별된다.  

대신 고유 키는 전통적인 디렉토리 구조를 모방하고 있어,  
계층 구조처럼 슬래시를 포함한 키를 가질 수 있다.

prefix는 접두사로, 전통적인 디렉토리 입장으로 보자면 '특정 폴더'로 정의할 수 있을 것이다.  

예를 들어  
로컬 컴퓨터에 /Users/Desktop/file.txt 라는 파일이 있다고 생각해보자.  
file.txt는 Users 디렉토리 안의 Desktop 안에 위치하고 있으며,  
따라서 file.txt의 경로는 /Users/Desktop/file.txt로 정의된다.  

예시를 이어 들어보면  
S3에도 /Users/Desktop/file.txt 라는 파일이 있다고 생각해보자.  
전통적 구조와 달리 이 파일은 디렉토리에 담겨있는 게 아니라  
"/Users/Desktop/file.txt" 이라는 키를 가진 파일 자체로 S3에 저장되어있다.  
따라서 디렉토리를 하나씩 들어갈 수는 없지만, 해당 경로로 파일에는 접근할 수 있는 것이다.  

여기서 prefix는 "/Users/Desktop/"이 될 것이다.  

Prefix의 용도는  
전통적인 directory 구조에서 "특정 폴더 안의 파일"만을 찾는다거나 "특정 폴더를 삭제" 하는 등의 역할을  
대신 해줄 수 있게 해준다.  

이해가 안된다면 어렵게 생각하지 말고, 그냥 "파일이 위치한 폴더"로 이해해도 큰 무리는 없을 것 같다.  
하지만 이해함.  

prefix 는 boto3.client() 방법에서만 사용이 가능하다.  
resource 방법으로는 사용이 불가능하다.  

In [ ]:
# s3 클라이언트 선언

s3_client = boto3.client('s3', aws_access_key_id = aws_access_key_id,
                         aws_secret_access_key = aws_secret_access_key)

In [85]:
# 버킷에 업로드 되어있는 파일 확인

res = s3_client.list_objects_v2(Bucket = bucket_name)

for obj in res['Contents']:
    print(obj['Key'])

앞으로의 프로젝트.xlsx


In [81]:
# 버킷에 파일 업로드 (방법 2)

prefix = '/test/sqls/'
local_file = '/Users/jongya/Desktop/antifragile.sql'

s3_client.upload_file(local_file, bucket_name, prefix + 'test3.sql')
s3_client.upload_file(local_file, bucket_name, prefix + 'test4.sql')
s3_client.upload_file(local_file, bucket_name, prefix + 'test5.sql')
s3_client.upload_file(local_file, bucket_name, prefix + 'test6.sql')

In [84]:
# /test/sqls/ 접두사를 가지는 파일 모두 삭제

buckets = s3.Bucket(name = bucket_name)

for i in buckets.objects.all():
    if prefix in i.key:
        s3_client.delete_object(Bucket = bucket_name, Key = i.key)